In [1]:
# data manipulation
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re
import numpy as np
import pandas as pd
import warnings 
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

# files
import os
import pathlib

# plotting
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.dates as mdates
import seaborn as sns

# web app
import streamlit as st

####################

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
# Clustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
# Prediction
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
## models
from sklearn.linear_model import LinearRegression # ols
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
## evaluation
from sklearn.model_selection import cross_val_score, KFold
from sklearn.base import BaseEstimator, RegressorMixin

In [2]:
# Set the path to the directory containing the Excel files
dir_path = r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Fantacalcio\Dati Fantacalcio.it\Statistiche"
# Get a list of all the files in the directory
file_list = os.listdir(dir_path)
# Create a list of dfs
df_list = []
# Loop through the files and read each one
for file_name in file_list:
    if file_name.endswith('.xlsx'):  # Check if file is a .xlsx file
        file_path = os.path.join(dir_path, file_name)
        df = pd.read_excel(file_path, skiprows=1)
        df["Anno"] = datetime.strptime("20" + re.findall("[0-9]+", (pd.read_excel(file_path, nrows=1).columns[0]))[1], "%Y")
        df_list.append(df)
data = pd.concat(df_list,axis=0,ignore_index=True)

# Change the players' name into the same format
data.Nome = data.Nome.apply(lambda x: x.lower().title())

# Combine the two different names for games to vote
data.Pv = data.Pg.combine_first(data.Pv)
data.drop("Pg", axis = "columns", inplace=True)
# Combine the two different names for Fantamedia
data.Mf = data.Mf.combine_first(data.Fm)
data.drop("Fm", axis = "columns", inplace=True)

# Drop the specific role column (Rm)
data.drop("Rm", axis = "columns", inplace=True)

# Reorder and rename columns
data = data.reindex(columns=["Id","R","Nome","Squadra","Anno","Mv","Mf","Gf","Gs","Rp","Rc","R+","R-","Ass","Amm","Esp","Au","Pv"])
data.columns = ["Id","R","Nome","Squadra","Anno","Media voto","Media FantaVoto","Gol fatti","Gol subiti","Rigori parati","Rigori calciati","Rigori segnati","Rigori sbagliati","Assist","Ammonizioni","Espulsioni","Autogol","Partite a voto"]

# Add the quotes
Qdir_path = r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Fantacalcio\Dati Fantacalcio.it\Quote"
Qfile_list = os.listdir(Qdir_path)
Qdf_list = []
# Loop through the files and read each one
c = 0
for Qfile_name in Qfile_list:
    if Qfile_name .endswith('.xlsx'):  # Check if file is a .xlsx file
        Qfile_path = os.path.join(Qdir_path, Qfile_name)
        Qdf = pd.read_excel(Qfile_path, skiprows=1)
        Qdf["Anno"] = datetime.strptime("20" + re.findall("[0-9]+", (pd.read_excel(Qfile_path, nrows=1).columns[0]))[1], "%Y")
        Qdf_list.append(Qdf)
quotes = pd.concat(Qdf_list,axis=0,ignore_index=True)

# Join quotes with data
withQuotes = data.merge(quotes, how="inner", on=['Nome', 'Anno'])
withQuotes.drop([Qcol for Qcol in withQuotes.columns if Qcol.endswith("_y")], axis=1, inplace=True) # drop duplicates columns from merging
# %%
withQuotes.columns = ['Id', 'Ruolo', 'Nome', 'Squadra', 'Anno', 'Media voto','Media FantaVoto', 'Gol fatti',
                      'Gol subiti', 'Rigori parati','Rigori calciati','Rigori segnati', 'Rigori sbagliati',
                      'Assist','Ammonizioni', 'Espulsioni', 'Autogol','Partite a voto', 'RM', 'Quota attuale',
                      'Quota iniziale', 'Differenza', 'Qt.A M', 'Qt.I M', 'Diff.M', 'FVM', 'FVM M'] # change columns names
withQuotes["FantaVoto/Quota iniziale"] = withQuotes["Media FantaVoto"] / withQuotes["Quota iniziale"]
withQuotes["Voto/Quota iniziale"] = withQuotes["Media voto"] / withQuotes["Quota iniziale"]

# Clean columns names
withQuotes = withQuotes.reindex(columns=['Ruolo', 'Nome', 'Squadra', 'Anno', 'Media voto',
       'Media FantaVoto','Quota iniziale',"FantaVoto/Quota iniziale","Voto/Quota iniziale",'Gol fatti', 'Gol subiti', 'Rigori parati',
       'Rigori calciati', 'Rigori segnati', 'Rigori sbagliati', 'Assist',
       'Ammonizioni', 'Espulsioni', 'Autogol', 'Partite a voto',
       'Quota attuale', 'Differenza tra quote','Id'])
# Crowd Differenza tra quote
withQuotes["Differenza tra quote"] = withQuotes["Quota iniziale"] - withQuotes["Quota attuale"]

In [ ]:
# withQuotes.to_csv(r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Fantacalcio\Dataframe con quote.csv")

In [3]:
# Different names = 1604
len(withQuotes.Nome.unique())
# Different Ids = 1603
len(withQuotes.Id.unique())
# maybe one name changed during the years so it's better to use Id

1603

# Model

Preprocessing

In [4]:
withQuotes_20plus_noGK = withQuotes[(withQuotes["Partite a voto"] > 20) & (withQuotes.Ruolo != "P")].reset_index(drop=True)

In [5]:
# Add the target column "Media FantaVoto (t+1)"
# 1031 observations
prediction_df = pd.DataFrame(columns=['Ruolo', 'Media FantaVoto', 'FantaVoto/Quota iniziale', 'Gol fatti', 'Rigori calciati','Rigori segnati', 'Rigori sbagliati', 'Assist', 'Ammonizioni','Espulsioni', 'Autogol', 'Partite a voto', 'Quota attuale','Differenza tra quote','Quota iniziale (t+1)','Media FantaVoto (t+1)'])
for i, record in withQuotes_20plus_noGK.iterrows():
    name = record.Nome
    year = record.Anno
    next_year_row = withQuotes_20plus_noGK[(withQuotes_20plus_noGK.Nome == name) & (withQuotes_20plus_noGK.Anno == year + relativedelta(years=1))]
    if len(next_year_row) > 0:
        next_year_row = next_year_row.rename(columns={'Quota iniziale': 'Quota iniziale (t+1)', 'Media FantaVoto': 'Media FantaVoto (t+1)'}) # change next year columns names
        this_year_columns = ['Ruolo', 'Media FantaVoto', 'FantaVoto/Quota iniziale', 'Gol fatti', 'Rigori calciati','Rigori segnati', 'Rigori sbagliati', 'Assist', 'Ammonizioni','Espulsioni', 'Autogol', 'Partite a voto', 'Quota attuale','Differenza tra quote']
        this_year_values = record[this_year_columns].to_frame().T.reset_index(drop=True)
        next_year_values = next_year_row[['Quota iniziale (t+1)','Media FantaVoto (t+1)']].reset_index(drop=True)
        prediction_row = pd.concat([this_year_values, next_year_values], axis=1)
        prediction_df = prediction_df.append(prediction_row, ignore_index=True)
prediction_df

,Ruolo,Media FantaVoto,FantaVoto/Quota iniziale,Gol fatti,Rigori calciati,Rigori segnati,Rigori sbagliati,Assist,Ammonizioni,Espulsioni,Autogol,Partite a voto,Quota attuale,Differenza tra quote,Quota iniziale (t+1),Media FantaVoto (t+1)
0,D,6.08,0.675556,1,0,0,0,1,7,0,0,27.0,10,-1,9,5.93
1,D,6.58,0.8225,4,0,0,0,0,3,0,0,36.0,15,-7,13,6.45
2,D,6.12,0.68,1,0,0,0,1,10,0,0,36.0,10,-1,9,6.06
3,D,6.76,0.563333,2,0,0,0,3,5,1,0,22.0,11,1,15,6.42
4,C,6.66,0.512308,3,0,0,0,4,4,0,0,35.0,17,-4,16,6.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,A,6.5,0.928571,7,0,0,0,0,5,1,0,30.0,14,-7,13,6.45
1027,A,6.25,0.520833,2,0,0,0,3,1,0,0,26.0,11,1,11,5.78
1028,A,6.27,0.57,3,0,0,0,0,1,0,0,24.0,10,1,9,7.06
1029,A,6.08,0.467692,6,0,0,0,0,10,0,1,36.0,10,3,10,5.80


In [6]:
# one-hot encoding for Ruolo
dummies = pd.get_dummies(prediction_df.Ruolo).iloc[:,0:2] # dummies for Attaccante and Centrocampista to avoid dummy trap
prediction_df.drop("Ruolo", axis=1, inplace=True)
prediction_df = pd.concat([dummies, prediction_df], axis=1)
# cast as float
prediction_df = prediction_df.astype('float64')

In [7]:
# feature augmentation
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False) # initiate polinomial transformer
X = prediction_df.loc[:, ~prediction_df.columns.isin(["Media FantaVoto (t+1)"])]
y = prediction_df["Media FantaVoto (t+1)"]
X_quad = poly.fit_transform(X) # create all the quadratic interactions
X_quad = pd.DataFrame(X_quad, columns=poly.get_feature_names_out(input_features=X.columns)) # Create a new DataFrame with the quadratic interaction features
prediction_df = pd.concat((X_quad, y), axis=1)

In [8]:
prediction_df.columns[0:16] # X without poly expansion

Index(['A', 'C', 'Media FantaVoto', 'FantaVoto/Quota iniziale', 'Gol fatti',
       'Rigori calciati', 'Rigori segnati', 'Rigori sbagliati', 'Assist',
       'Ammonizioni', 'Espulsioni', 'Autogol', 'Partite a voto',
       'Quota attuale', 'Differenza tra quote', 'Quota iniziale (t+1)'],
      dtype='object')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(prediction_df.loc[:, ~prediction_df.columns.isin(["Media FantaVoto (t+1)"])], prediction_df["Media FantaVoto (t+1)"], test_size=0.3, random_state=20)

EDA

In [10]:
# sns.pairplot(pd.concat((X_train, y_train), axis=1))

OLS

In [11]:
# OLS with statsmodel
X_train_for_ols = sm.add_constant(X_train)
X_test_for_ols = sm.add_constant(X_test)
ols = sm.OLS(y_train, X_train_for_ols)
ols_fit = ols.fit()
# ols_fit.summary()

In [12]:
# OLS with sklearn
ols = LinearRegression()
ols_fit = ols.fit(X_train, y_train)

In [13]:
# ols_fit.mse_resid
mean_squared_error(y_train, ols_fit.predict(X_train)), mean_squared_error(y_test, ols_fit.predict(X_test))

(0.15123973791142215, 0.2601014234746098)

OLS with only significant variables

In [14]:
p_values = ols_fit.pvalues
significant_variables = p_values[p_values <= 0.05].index
# train and test with only significant variables
X_train_significant = X_train_for_ols[significant_variables]
X_train_significant = sm.add_constant(X_train_significant)
X_test_significant = X_test_for_ols[significant_variables]
X_test_significant = sm.add_constant(X_test_significant)
# ols
ols_significant = sm.OLS(y_train, X_train_significant)
ols_fit_significant = ols_significant.fit()
ols_fit_significant.summary()

AttributeError: 'LinearRegression' object has no attribute 'pvalues'

In [ ]:
# ols_fit_significant.mse_resid
mean_squared_error(y_train, ols_fit_significant.predict(X_train_significant)), mean_squared_error(y_test, ols_fit_significant.predict(X_test_significant))

In [ ]:
# training errors
(y_train - ols_fit.predict(X_train_for_ols)).hist(bins=100)

In [ ]:
# test errors
(y_test - ols_fit.predict(X_test_for_ols)).hist(bins=100)

In [ ]:
# (test["Media FantaVoto (t+1)"] - ols_test_pred)[(test["Media FantaVoto (t+1)"] - ols_test_pred) > 2] # index 87
# test.loc[87]
# # withQuotes_20plus_noGK[(withQuotes_20plus_noGK.Ruolo == "A") & (withQuotes_20plus_noGK["Media FantaVoto"] == 6.69) & (withQuotes_20plus_noGK["Ammonizioni"] == 7)]
# test["Media FantaVoto (t+1)"].sort_values(ascending = False)
# A very high error is made on Mertens (underestimated by 2.6) so i am gonna check how does the error change with higher Media FantaVoto (t+1)
plt.scatter(x = y_test, y = (y_test - ols_fit.predict(X_test_for_ols)))

XGBoost

In [ ]:
# xgb_model = xgb.XGBRegressor(n_estimators=1000000, eta=0.01, colsample_bytree=np.sqrt(len(X_train.columns))/len(X_train.columns))
# xgb_fit = xgb_model.fit(X_train, y_train)
# xgb_fit.save_model(r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Fantacalcio\xgb1.model")

In [ ]:
xgb_fit = xgb.Booster()
xgb_fit.load_model(r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Fantacalcio\xgb.model") # big model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(prediction_df.loc[:, ~prediction_df.columns.isin(["Media FantaVoto (t+1)"])], prediction_df["Media FantaVoto (t+1)"], test_size=0.3, random_state=0)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
xgb_pred = xgb_fit.predict(dtest)
mean_squared_error(y_train, xgb_fit.predict(dtrain)), mean_squared_error(y_test, xgb_pred) # 0.2027

In [ ]:
# too long to compute 
# scores = cross_val_score(xgb_model, prediction_df.loc[:, ~prediction_df.columns.isin(["Media FantaVoto (t+1)"])], prediction_df["Media FantaVoto (t+1)"], cv=5, scoring='neg_mean_squared_error') # 0.22
# scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error') # 0.235
# mean_mse = -np.mean(scores)
# mean_mse

In [ ]:
plt.scatter(x = y_test, y = (y_test - xgb_pred), s= 10)
plt.plot((5.5,9.5),(0.5,0.5), c = "green")
plt.plot((5.5,9.5),(-0.5,-0.5), c = "green")
rectangle = patches.Rectangle((5.5,(0.5)), 4, -1, edgecolor='green', facecolor='green', alpha = 0.12)
plt.gca().add_patch(rectangle)
plt.gca().set_xlabel('Media FantaVoto (t+1)')
plt.gca().set_ylabel('Errore del modello')
plt.gca().set_title('Correlazione FantaVoto-Errore');

# Circa 20 di 310 vengono sbagliati per più di mezzo punto

Tree

In [ ]:
tree = DecisionTreeRegressor(random_state=42)
tree = tree.fit(X_train, y_train)
tree_pred = tree.predict(X_test)
mean_squared_error(y_test, tree_pred)

In [ ]:
plt.scatter(x = y_test, y = (y_test - tree_pred))

# Prediction for 2023/24
#### Dataframe

In [ ]:
# Remove goalkeepers
withQuotes_noGK = withQuotes[withQuotes["Ruolo"] != "P"]

# Create 2023 df
df_2023 = withQuotes_noGK[withQuotes_noGK.Anno == datetime(2023,1,1)] # 475 players

# Create new quotes df
newQuotes_path = r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Fantacalcio\Dati Fantacalcio.it\Nuove quote\Quotazioni_Fantacalcio_Stagione_2023_24.xlsx"
newQuotes = pd.read_excel(newQuotes_path, skiprows=1)
newQuotes["Anno"] = datetime.strptime("20" + re.findall("[0-9]+", (pd.read_excel(newQuotes_path, nrows=1).columns[0]))[1], "%Y")
newQuotes.columns = ['Id', 'Ruolo', 'RM', 'Nome', 'Squadra', 'Quota attuale ', 'Quota iniziale', 'Differenza', 'Qt.A M',
        'Qt.I M', 'Diff.M', 'FVM', 'FVM M', 'Anno'] # change names of the variables of interest
newQuotes.drop(['RM','Qt.A M', 'Qt.I M', 'Diff.M', 'FVM', 'FVM M'], inplace=True, axis=1) # drop useless columns
newQuotes.rename(columns={"Quota iniziale": "Quota iniziale (t+1)"}, inplace=True)
newQuotes = newQuotes[["Id", "Quota iniziale (t+1)"]] # keep only the Id for the merge and the Quota iniziale (t+1)
################# 508 players

# Merge 2023 df and new quotes
new_prediction_df = pd.merge(df_2023, newQuotes, how = "inner", on = "Id")
roles = new_prediction_df.Ruolo
names = new_prediction_df.Nome
################# 336 players

# keep only the needed columns for predictions (numerical ones and Ruolo)
new_prediction_df = new_prediction_df[['Ruolo', 'Media FantaVoto', 'FantaVoto/Quota iniziale', 'Gol fatti', 'Rigori calciati','Rigori segnati', 'Rigori sbagliati', 'Assist', 'Ammonizioni','Espulsioni', 'Autogol', 'Partite a voto', 'Quota attuale','Differenza tra quote', 'Quota iniziale (t+1)']]
### clustering_df is created keeping only the pitch-performance metrics
clustering_df = new_prediction_df[['Gol fatti', 'Rigori calciati','Rigori segnati', 'Rigori sbagliati', 'Assist', 'Ammonizioni','Espulsioni', 'Autogol', 'Partite a voto']]

### one-hot encoding for Ruolo
dummies = pd.get_dummies(new_prediction_df.Ruolo).iloc[:,0:2] # dummies for Attaccante and Centrocampista to avoid dummy trap
new_prediction_df.drop("Ruolo", axis=1, inplace=True)
new_prediction_df = pd.concat([dummies, new_prediction_df], axis=1)

### cast as float
new_prediction_df = new_prediction_df.astype('float64')

## feature augmentation
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False) # initiate polinomial transformer
X = new_prediction_df.loc[:, ~new_prediction_df.columns.isin(["Media FantaVoto (t+1)"])]
X_quad = poly.fit_transform(X) # create all the quadratic interactions
new_prediction_df = pd.DataFrame(X_quad, columns=poly.get_feature_names_out(input_features=X.columns)) # Create a new DataFrame with the quadratic interaction features
new_prediction_df

#### Prediction

In [ ]:
dnew = xgb.DMatrix(new_prediction_df)
new_pred_values = xgb_fit.predict(dnew)
plt.hist(new_pred_values)
new_pred_names = pd.concat([names, pd.Series(new_pred_values, name='Media FantaVoto 23/24 (previsione)'), new_prediction_df["Quota iniziale (t+1)"]], axis=1)
new_pred_names.rename(columns={"Quota iniziale (t+1)": "Quota"}, inplace=True)
new_pred_names["Rapporto FantaVoto/Quota"] = new_pred_names['Media FantaVoto 23/24 (previsione)']/new_pred_names["Quota"]
new_pred_names = pd.concat((new_pred_names, roles), axis=1)
new_pred_names

In [ ]:
new_pred_names.sort_values('Media FantaVoto 23/24 (previsione)', ascending=False).head(20)
# new_pred_names.to_csv(r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Fantacalcio\Previsioni 23-24.csv")

# CLUSTERING

In [ ]:
# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(clustering_df)

In [ ]:
# K means
k_values = range(2, 11)
silhouette_scores = []
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(data_scaled)
    silhouette_scores.append(silhouette_score(data_scaled, cluster_labels))
best_k = k_values[np.argmax(silhouette_scores)]

plt.plot(k_values, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Different K Values')
plt.show()

In [ ]:
# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_scaled)

In [ ]:
# Get the explained variance ratios
explained_variance = pca.explained_variance_ratio_

# Create the biplot
fig, ax = plt.subplots(figsize=(8, 6), dpi=100)
# ax.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.5)

# Add variable loadings
for i in range(clustering_df.shape[1]): # for each variable index
    ax.arrow(0, 0, pca.components_[0][i], pca.components_[1][i], head_width=0.01, head_length=0.01, linewidth=0.5, color='red')
    ax.text(pca.components_[0][i] + 0.03, pca.components_[1][i] + 0.03, clustering_df.columns[i], color='black', ha='center', va='center')

# Set plot title and labels
ax.set_title('PCA loadings plot')
ax.set_xlabel('PC1 ({:.2%})'.format(explained_variance[0]))
ax.set_ylabel('PC2 ({:.2%})'.format(explained_variance[1]))

# Show the plot
plt.show()

In [ ]:
# K means over PCA results
k_values = range(2, 11)
silhouette_scores = []
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(data_scaled)
    silhouette_scores.append(silhouette_score(pca_result, cluster_labels))
best_k = k_values[np.argmax(silhouette_scores)]

## plot silhouette score
plt.plot(k_values, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Different K Values')
plt.show()

In [ ]:
# Perform K-means clustering with the best K value
kmeans = KMeans(n_clusters=2, random_state=42)
cluster_labels = kmeans.fit_predict(pca_result)

# Create the biplot with colored clusters
fig, ax = plt.subplots(figsize=(10, 7), dpi=120)
for i in range(2):
    ax.scatter(pca_result[cluster_labels == i, 0], pca_result[cluster_labels == i, 1], label=f'Cluster {i}')

# Add text labels for each data point
for i, label in enumerate(new_pred_names.Nome):
    ax.text(pca_result[i, 0] + 0.05, pca_result[i, 1] + 0.05, s=label)
ax.legend()

# Add variable loadings (same as before)
# left empty because it messed the plot

# Set plot title and labels
ax.set_title('PCA results with Clusters')
ax.set_xlabel('PC1 ({:.2%})'.format(explained_variance[0]))
ax.set_ylabel('PC2 ({:.2%})'.format(explained_variance[1]))

# Show the plot
plt.show()

In [ ]:
new_pred_names = pd.concat((new_pred_names, pd.Series(cluster_labels, name="Cluster")), axis=1) # execute this line only once
new_pred_names.Cluster = new_pred_names.Cluster.map({1: "Malus taker", 0: "Bonus taker"})
new_pred_names

In [ ]:
new_pred_names.groupby("Cluster").mean()

In [ ]:
new_pred_names[(new_pred_names.Ruolo == "C") & (new_pred_names.Cluster == "Bonus taker")]

# Graphical analysis

In [ ]:
# Create variables user selection
nome1 = st.selectbox("Giocatore 1", new_pred_names.Nome.unique()) #(use only the names of players for 2023/24)
nome2 = st.selectbox("Giocatore 2", new_pred_names.Nome.unique()) #(use only the names of players for 2023/24)
metrica = st.selectbox("Metrica", withQuotes.columns[4:withQuotes.shape[1]-2]) # from Media voto to Quota attuale (left out Differenza tra quote)

# Mini timeseries
player1 = withQuotes[withQuotes.Nome == nome1][["Anno", metrica]]
player2 = withQuotes[withQuotes.Nome == nome2][["Anno", metrica]]

# Merged df of both players metrics
years = sorted(set(player1['Anno']) | set(player2['Anno'])) # Determine the unique years from both dataframes
merged = pd.DataFrame({'Anno': years}) # Create a new dataframe with the unique years
merged = merged.merge(player1, on='Anno', how='left') # Merge player1 data
merged = merged.merge(player2, on='Anno', how='left') # Merge player2 data
merged.columns = ['Anno', nome1, nome2] # Rename the columns

# Set "Anno" as index
merged = merged.set_index('Anno')

# Plot
## Create a figure and axes
fig, ax = plt.subplots()
ax.plot(merged.index, merged[nome1], label=nome1, marker = '.')
ax.plot(merged.index, merged[nome2], label=nome2, marker = '.')
date_form = mdates.DateFormatter("%Y") # Set x-axis tick formatter to display only the year
ax.xaxis.set_major_formatter(date_form)
ax.set_xlabel('Year')
ax.set_ylabel('Value')
#ax.set_ylim(bottom=min(withQuotes[metrica]), top=max(withQuotes[metrica]))

ax.legend()
fig.tight_layout()
## Display the chart using Streamlit
st.pyplot(fig)